# Analyse de Clustering avec GMM sur OpenFoodFacts
### Table des matières
1. [Chargement et préparation des données](#1),
2. [Prétraitement et nettoyage](#2),
3. [Sélection des features pour le clustering](#3),
4. [Préparation pour GMM](#4),
5. [Modélisation GMM](#5),
6. [Analyse des clusters](#6),
7. [Visualisation des résultats](#7)

## Chargement et préparation des données


In [1]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

from scripts.features.processing import (
    process_numeric_columns,
    process_categorical_columns,
    detect_ordinal_nature
)
from scripts.features.selection import (
    analyze_and_select_features,
    analyze_feature_importance,
    select_relevant_features
)
from scripts.data.analysis import analyze_data_quality

# Chargement des données avec limitation à 300000 lignes
from scripts.data_handling.loader import load_data
df = pd.read_csv('../data/en.openfoodfacts.org.products.csv', 
                 sep='\t',
                 encoding='utf-8',
                 low_memory=False,
                 on_bad_lines='skip',
                 nrows=300000)

In [ ]:
# Paramètres de prétraitement
MAX_CATEGORIES = 30  # Nombre maximum de catégories pour les variables catégorielles
MIN_UNIQUE_RATIO = 0.01  # Ratio minimum de valeurs uniques pour les variables numériques
MISSING_THRESHOLD = 0.5  # Seuil maximum de valeurs manquantes acceptables

# Analyse initiale de la qualité des données
print("1. Analyse de la qualité des données initiales :")
quality_report = analyze_data_quality(df)
print(f"\nNombre de lignes dupliquées : {quality_report['duplicates']}")
print("\nDistribution des valeurs manquantes :")
display(quality_report['missing_values'])

# Prétraitement et sélection automatique des features
print("\n2. Détection et prétraitement automatique des variables :")
df_processed, feature_info = analyze_and_select_features(
    df,
    max_categories=MAX_CATEGORIES,
    min_unique_ratio=MIN_UNIQUE_RATIO,
    missing_threshold=MISSING_THRESHOLD
)

# Affichage des résultats du prétraitement
print("\n3. Résumé des variables détectées :")
print(f"Variables numériques : {len(feature_info['feature_types']['numeric'])}")
print(f"Variables ordinales : {len(feature_info['feature_types']['ordinal'])}")
print(f"Variables nominales : {len(feature_info['feature_types']['nominal'])}")

# Affichage des optimisations de types de données
if feature_info['downcasted_columns']:
    print("\n4. Optimisations des types de données :")
    for col, old_type, new_type in feature_info['downcasted_columns']:
        print(f"- {col}: {old_type} → {new_type}")

# Affichage des colonnes filtrées
if feature_info['dropped_columns']:
    print("\n5. Colonnes filtrées :")
    for col, reason in feature_info['dropped_columns']:
        print(f"- {col} (raison: {reason})")

# Analyse des corrélations entre variables numériques
print("\n6. Analyse des corrélations entre variables numériques :")
correlations = analyze_correlations(df_processed, threshold=0.7, plot=True)
if correlations['strong_correlations']:
    print("\nPaires de variables fortement corrélées :")
    for corr in correlations['strong_correlations']:
        print(f"- {corr['var1']} - {corr['var2']}: {corr['correlation']:.2f}")

# Sauvegarde du DataFrame prétraité
df = df_processed.copy()
print(f"\nDimensions finales du DataFrame : {df.shape}")